# TCP/IP Client and Server

Sockets can be configured to act as a server and listen for incoming messages, or connect to other applications as a client. After both ends of a TCP/IP socket are connected, communication is bi-directional.

## Echo Server

This sample program, based on the one in the standard library documentation, receives incoming messages and echos them back to the sender. It starts by creating a TCP/IP socket, then bind() is used to associate the socket with the server address. In this case, the address is localhost, referring to the current server, and the port number is 10000.

Calling listen() puts the socket into server mode, and accept() waits for an incoming connection. The integer argument is the number of connections the system should queue up in the background before rejecting new clients. This example only expects to work with one connection at a time.

accept() returns an open connection between the server and client, along with the address of the client. The connection is actually a different socket on another port (assigned by the kernel). Data is read from the connection with recv() and transmitted with sendall().

When communication with a client is finished, the connection needs to be cleaned up using close(). This example uses a try:finally block to ensure that close() is always called, even in the event of an error.

## Echo Client¶

The client program sets up its socket differently from the way a server does. Instead of binding to a port and listening, it uses connect() to attach the socket directly to the remote address.



After the connection is established, data can be sent through the socket with sendall() and received with recv(), just as in the server. When the entire message is sent and a copy received, the socket is closed to free up the port.

## Client and Server Together

The client and server should be run in separate terminal windows, so they can communicate with each other. The server output shows the incoming connection and data, as well as the response sent back to the client.

The client output shows the outgoing message and the response from the server.

In [1]:
! python3 socket_echo_client.py

connecting to localhost port 10000
sending b'This is the message.  It will be repeated.'
received b'This is the mess'
received b'age.  It will be'
received b' repeated.'
closing socket


## Easy Client Connections

TCP/IP clients can save a few steps by using the convenience function create_connection() to connect to a server. The function takes one argument, a two-value tuple containing the address of the server, and derives the best address to use for the connection.

create_connection() uses getaddrinfo() to find candidate connection parameters, and returns a socket opened with the first configuration that creates a successful connection. The family, type, and proto attributes can be examined to determine the type of socket being returned.

In [2]:
! python3 socket_echo_client_easy.py

Family  : AF_INET
Type    : SOCK_STREAM
Protocol: IPPROTO_TCP

sending b'This is the message.  It will be repeated.'
received b'This is the mess'
received b'age.  It will be'
received b' repeated.'
closing socket


In [3]:
import socket

def get_constants(prefix):
    """Create a dictionary mapping socket module
    constants to their names.
    """
    return {
        getattr(socket, n): n
        for n in dir(socket)
        if n.startswith(prefix)
    }


families = get_constants('AF_')
types = get_constants('SOCK_')
protocols = get_constants('IPPROTO_')

In [4]:
families

{<AddressFamily.AF_UNSPEC: 0>: 'AF_UNSPEC',
 <AddressFamily.AF_UNIX: 1>: 'AF_UNIX',
 <AddressFamily.AF_INET: 2>: 'AF_INET',
 <AddressFamily.AF_SNA: 11>: 'AF_SNA',
 12: 'AF_DECnet',
 <AddressFamily.AF_APPLETALK: 16>: 'AF_APPLETALK',
 <AddressFamily.AF_ROUTE: 17>: 'AF_ROUTE',
 <AddressFamily.AF_LINK: 18>: 'AF_LINK',
 <AddressFamily.AF_IPX: 23>: 'AF_IPX',
 <AddressFamily.AF_INET6: 30>: 'AF_INET6',
 <AddressFamily.AF_SYSTEM: 32>: 'AF_SYSTEM'}

In [5]:
types

{<SocketKind.SOCK_STREAM: 1>: 'SOCK_STREAM',
 <SocketKind.SOCK_DGRAM: 2>: 'SOCK_DGRAM',
 <SocketKind.SOCK_RAW: 3>: 'SOCK_RAW',
 <SocketKind.SOCK_RDM: 4>: 'SOCK_RDM',
 <SocketKind.SOCK_SEQPACKET: 5>: 'SOCK_SEQPACKET'}

In [6]:
protocols

{0: 'IPPROTO_IP',
 1: 'IPPROTO_ICMP',
 2: 'IPPROTO_IGMP',
 3: 'IPPROTO_GGP',
 4: 'IPPROTO_IPV4',
 6: 'IPPROTO_TCP',
 8: 'IPPROTO_EGP',
 12: 'IPPROTO_PUP',
 17: 'IPPROTO_UDP',
 22: 'IPPROTO_IDP',
 29: 'IPPROTO_TP',
 36: 'IPPROTO_XTP',
 41: 'IPPROTO_IPV6',
 43: 'IPPROTO_ROUTING',
 44: 'IPPROTO_FRAGMENT',
 46: 'IPPROTO_RSVP',
 47: 'IPPROTO_GRE',
 50: 'IPPROTO_ESP',
 51: 'IPPROTO_AH',
 58: 'IPPROTO_ICMPV6',
 59: 'IPPROTO_NONE',
 60: 'IPPROTO_DSTOPTS',
 63: 'IPPROTO_HELLO',
 77: 'IPPROTO_ND',
 80: 'IPPROTO_EON',
 103: 'IPPROTO_PIM',
 108: 'IPPROTO_IPCOMP',
 132: 'IPPROTO_SCTP',
 255: 'IPPROTO_RAW',
 256: 'IPPROTO_MAX'}

## Choosing an Address for Listening

It is important to bind a server to the correct address, so that clients can communicate with it. The previous examples all used 'localhost' as the IP address, which limits connections to clients running on the same server. Use a public address of the server, such as the value returned by gethostname(), to allow other hosts to connect. This example modifies the echo server to listen on an address specified via a command line argument.

A similar modification to the client program is needed before the server can be tested.

After starting the server, the netstat command shows it listening on the address for the named host.

In [7]:
! ping -t 3 MementoMori

PING mementomori (127.0.0.1): 56 data bytes
64 bytes from 127.0.0.1: icmp_seq=0 ttl=64 time=0.082 ms
64 bytes from 127.0.0.1: icmp_seq=1 ttl=64 time=0.066 ms
64 bytes from 127.0.0.1: icmp_seq=2 ttl=64 time=0.085 ms

--- mementomori ping statistics ---
3 packets transmitted, 3 packets received, 0.0% packet loss
round-trip min/avg/max/stddev = 0.066/0.078/0.085/0.008 ms


In [8]:
! netstat -an | grep 10001

tcp4       0      0  127.0.0.1.10001        *.*                    LISTEN     


In [9]:
! python3 socket_echo_client_explicit.py MementoMori

connecting to MementoMori port 10001
sending b'This is the message.  It will be repeated.'
received b'This is the mess'
received b'age.  It will be'
received b' repeated.'


Many servers have more than one network interface, and therefore more than one IP address. Rather than running separate copies of a service bound to each IP address, use the special address INADDR_ANY to listen on all addresses at the same time. Although socket defines a constant for INADDR_ANY, it is an integer value and must be converted to a dotted-notation string address before it can be passed to bind(). As a shortcut, use “0.0.0.0” or an empty string ('') instead of doing the conversion.

To see the actual address being used by a socket, call its getsockname() method. After starting the service, running netstat again shows it listening for incoming connections on any address.

In [10]:
! netstat -an | grep 10002

tcp4       0      0  *.10002                *.*                    LISTEN     
